Libraries importation

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
import pandas as pd
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

Directories determination

In [40]:
train_data_dir = r'C:\Users\user\Desktop\Neuro\data\train'
val_data_dir = r'C:\Users\user\Desktop\Neuro\data\valid'

Data - normalization

In [41]:
# train data with augmentation
train_datagen = ImageDataGenerator(
    tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.18,
    zoom_range=0.18,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    channel_shift_range=6.5,
    fill_mode='nearest'
)

# Only normalization to val section
val_datagen = ImageDataGenerator(tf.keras.applications.efficientnet.preprocess_input)

Data - generators

In [42]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=48,
    class_mode='sparse'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size = 48,
    class_mode='sparse'
)

Found 6552 images belonging to 102 classes.
Found 818 images belonging to 102 classes.


Let's determine base model and freeze it to prevent training 

In [43]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

Build new layers and unfreeze last 40 layers of EffecientNet

In [44]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    layers.Dense(102, activation='softmax')
])

for layer in base_model.layers[-40:]:
    layer.trainable = True

Model compilation

In [45]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Model - training

In [46]:
history = model.fit(
    train_generator,
    epochs=6,  # Больше эпох
    validation_data=val_generator,
)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 122s 829ms/step - accuracy: 0.1216 - loss: 8.3253 - val_accuracy: 0.4352 - val_loss: 5.8335
Epoch 2/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 112s 813ms/step - accuracy: 0.5959 - loss: 5.4535 - val_accuracy: 0.8350 - val_loss: 3.7313
Epoch 3/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 111s 810ms/step - accuracy: 0.8547 - loss: 3.7060 - val_accuracy: 0.9218 - val_loss: 2.9577
Epoch 4/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 114s 834ms/step - accuracy: 0.9294 - loss: 2.8630 - val_accuracy: 0.9389 - val_loss: 2.4182
Epoch 5/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 112s 814ms/step - accuracy: 0.9537 - loss: 2.3297 - val_accuracy: 0.9499 - val_loss: 2.0027
Epoch 6/6
137/137 ━━━━━━━━━━━━━━━━━━━━ 111s 813ms/step - accuracy: 0.9588 - loss: 1.9237 - val_accuracy: 0.9572 - val_loss: 1.6631
